In [1]:
# Pipeline의 구조를 이해하기 쉽게 하기 위해 작성한 ipynb 파일입니다
from Pipeline import pipe_eval
pipe = pipe_eval(verbose = True) # verbose를 False로 하면 print가 되지 않습니다.
# collection_name = 'chrdb.db',
# persist_directory = 'DB',
# mode = 'NaverCloudEmb'
# topk = 5
# verbose = True : answer과 score을 보여줍니다. 보고싶지 않다면 False로 하면 됩니다.
# default로 지정돼 있습니다. 클래스 선언 때 건네주면 바뀝니다.
pipe.setup(model = 'GPT')
#                   가능한 모델 (default GPT)
#                   1. GPT
#                   2. Exaone
#                   3. Qwen14B
#                   4. Qwen32B

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
생성된 DB가 있어 로드합니다.


/data/ephemeral/home/clone/src/DB/chromadb_storing.py:105: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.db = Chroma(


Collection 'chrdb.db' loaded from 'DB/DB_NaverCloudEmb'.
Total documents in collection 'chrdb.db': 388


In [2]:
query = 'SK하이닉스의 2025~2027년 주주환원 정책에서 변경된 고정배당금과 추가 배당금의 활용 방안은 무엇인가요?'
# validation데이터셋의 첫번째 question
retrieval_result = pipe.Q(query, mode = 'ensemble')
# bm25, dpr, ensemble 가능 (default 'ensemble')


In [3]:
answer = pipe.A(query, retrieval_result)
# query 와 retrieval 결과를 받아 answer을 냅니다.


In [4]:
answer = pipe.QA(query, mode = 'ensemble')
# QA를 따로 내는 것이 아니라 한번에 정답을 얻을 수도 있습니다. 이후 정량평가에서는 QA를 사용하면 됩니다.

In [2]:
generated_answer, G_retrieval_score, G_generation_score = pipe.QA_eval(mode = 'ensemble', sampling = True)

# validation 데이터셋의 무작위 샘플을 가져와 G-eval로 평가할 수 있습니다.

page_content='카카오뱅크는 대출 갈아타기 시행 후 신용대출, 주택담보대출, 전월세보증금 대출에서 유입액이 유출액을 크게 웃돌고 있으며, 신규 상품 출시와 경쟁심화 기조로 인해 성장세가 이어질 것으로 예상된다.' metadata={'company_name': '카카오뱅크', 'dir_route': 'datas/카카오뱅크_한국신용평가(2024.09.13)', 'file_name': '카카오뱅크_한국신용평가(2024.09.13).pdf', 'id': 51, 'investment': '한국신용평가(2024.09.13)', 'original_content': '대 출 갈아타기 시행 이후(신용대출은 2023년 5월 시행, 주택담보대출 및 전월세보증금 대출은 2024 년부터 시행), 신용대출, 주택담보대출, 전월세보증금 대출 각각 유입금액이 유출 금액을 크게 상회하 고 있다. 지속적으로 신규 상품을 출시하고 있는 점, 은행의 경쟁심화 기조가 유리하게 작용하고 있는 점 등을 고려할 때, 동행의 성장추세는 당분간 지속될 것으로 판단한다.', 'page': 3, 'type': 'paragraph'}
page_content='카카오뱅크는 높은 브랜드 인지도와 편리한 모바일 앱을 바탕으로 개인고객을 많이 확보했으며, 특히 주택담보대출 상품 출시 후 고객 기반이 더욱 넓어졌다. 그 결과 대출채권 규모가 빠른 속도로 증가해 2024년 6월 말에는 42.7조 원에 달했고, 이는 전년 대비 37.8%, 올해 상반기 대비 10.6% 상승한 수치이다. 이러한 성장세는 주로 주택담보대출 분야에서의 성과 덕분이며, 전체 대출채권 증가액 중 7.9조 원이 주택담보대출 증가분이었다.' metadata={'company_name': '카카오뱅크', 'dir_route': 'datas/카카오뱅크_한국신용평가(2024.09.13)', 'file_name': '카

In [ ]:
eval_df = pipe.QA_eval(mode = 'ensemble', sampling = False) # sampling이 False일 경우, 모든 validation 데이터셋에 대해 평가를 진행합니다.
# 구현이 귀찮아서 일단 ./datas/g_eval_result_{model_name}.csv 경로에 저장되도록 하였습니다..

0it [00:00, ?it/s]

193it [53:34, 16.34s/it]

In [42]:
import pandas as pd
# retrieval G eval score과 generation G eval score의 점수 부분만 파싱합니다. (저장된 csv에는 전부 문자로 돼 있습니다.)

eval_df['generation_score'] = eval_df['generation_score'].apply(lambda x: x[x.rfind('/')-3:x.rfind('/')])
eval_df['retrieval_score'] = eval_df['retrieval_score'].apply(lambda x: x[x.rfind('/')-3:x.rfind('/')])
eval_df['generation_score'] = pd.to_numeric(eval_df['generation_score'], errors = 'coerce')
eval_df['retrieval_score'] = pd.to_numeric(eval_df['retrieval_score'], errors = 'coerce')

In [45]:
eval_df['generation_score'].describe()

count    264.000000
mean      20.265152
std        3.939388
min       10.000000
25%       18.000000
50%       20.000000
75%       24.000000
max       27.000000
Name: generation_score, dtype: float64